# Simulation

Goal: to run a simulation based on **real-time location** tracking of ambulances. The simulation itself may or may not be used. But I hope that the **modules/ideas** making up this simulation could end up being useful.

- `Clocked`
- `Case events` are randomly entered

Personal note: [Github Markdown Including YouTube Videos](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

More information written below. 

# Taking in Cases:
Okay, so now that I have clock, I want to implement reading in the cases that happen in "real" time. 

### Easy but False Simulation 

- One way I could do that is to read it from a file. Characteristics of the events would be:

    - `GPS Location`
    - `Severity`
    - `Time it comes in`

- Characteristics of the case that are unpredictable, but useful:

    - The `time` it takes to accomplish the case
    - The actual `travel time`

### Harder but Realistic Simulation

- The other thing I could do is to have the events happen `randomly`. This is not hard. 

- The last thing I could do is to `manually input` the events which is the most difficult since this would be a continuous program that reacts based on user inputs. It would probably use GRPCs. 

- The random events can be shaped by seeds, which means we may be able to `shape the way new events are initialized`. 

# Implementing the Simulation

- Ambulances should be timed out. When they are sent on cases, they are timed in. There is a `time t` the ambulance is on-case. When the time finishes, it returns to Idle.
- New cases are started when the `arguments` come in and the time matches its start date. The other way to start cases is based on real time: manually input it. 
- We can randomly generate the `timeline` of cases that will backbone the simulation.

# Data Structures (ABC)


### Ambulances

a dict of tuple ()

```
(1) n <-- number of ambulances
(2) ambulance [n]
(3) each ambulance --> <status, time on-case>

```

### Base Locations

array 

```
(1) base location [x] where x <-- number of bases which is 100 

```

### Case Locations

array

```
(1) case location [y] where y <-- number of possible demand points 
    for which we have travel times. 

```

# Pseudocode Algorithm

I want to implement the following now:

```
for each time unit (t):
    
    update all times (t [amb1 .. amb_n])
    
    if a new case should start, then start it 
    
    if an on-case should end, then end it
    
    update the set coverage and coverage change (coverage, delta)
    

```

In [17]:
import time
import sys
from enum import Enum

In [18]:
class Status(Enum):
    """ For ambulances """
    IDLE = 1
    ACTIVE = 2

In [19]:
class DataSource(Enum):
    """ There are at least three sources of events for the simulation. """
    RANDOM = 1

In [20]:
def generate_random_amb_event(ambulance_set, demand_set):
    """ Given the set of ambulances and the set of demand points, randomly generate an event """
    """ The return type should specify ambulance and demand point."""
    
    return (0,0)

In [21]:
def read_next_event(data_source):
    """ Reads the next event from data set or generates one. """
    
    if data_source is DataSource.RANDOM:
        (amb, demand) = generate_random_amb_event(ambulance_set, demand_set)
    
    pass

In [22]:
def change_state(amb_status, amb_event):
    """ Takes the ambulance status dictionary, and an event.
    If the event has 'occurred', then update amb_status and delete the event.  """    
    
    if type(amb_status) is not dict: raise Exception("Wrong type. ") 
    
    output = ""
    for ID in amb_status: output += ("Ambulance %i %s \n" %(ID, amb_status[ID]))
    print(output)
        
    pass

In [23]:
def timed_execute(run_time=1, speed_times=4, amb_status = None):
    """ Starts a clock """
    
    if amb_status == None: raise Exception ("param amb_status is None")
    if type(amb_status) is not dict: raise Exception ("amb_status wrong type")
    if type(run_time) is not int: raise Exception("run_time not int")
    if type(speed_times) is not int: raise Exception("speed_times not int")

    start = time.time()
    time_diff = 0

    while True:
        """ Every unit of time, update the next ambulance event, print, update time. """
        
        time.sleep(1/speed_times)
        print ("\rTime: " + str(time_diff))
        
        # Read the next event and then change the state:
        amb_event = read_next_event(DataSource.RANDOM)
        change_state(amb_status, amb_event)
        
        print()
        now = time.time()
        time_diff = (now - start)*speed_times
        time_diff = int(time_diff)
        if time_diff > run_time: break 
            
        

In [24]:
run_time = 2
speed = 2

ambulance_count = 8
amb_status = {} 
for ID in range(0, ambulance_count):
    amb_status[ID] = Status.IDLE

timed_execute(run_time=run_time, speed_times=speed, amb_status = amb_status)

Time: 0


NameError: name 'ambulance_set' is not defined